In [2]:
import numpy as np
import timeit

# Bitwise AND operation
setup_and = """
import numpy as np
a = np.array([1, 2, 3, 4, 5, 6, 7, 8])
b = np.array([1, 2, 3, 4, 5, 6, 7, 8])
"""

stmt_and = "a ^= b"

# Addition followed by modulo
setup_mod = """
import numpy as np
a = np.array([1, 2, 3, 4, 5, 6, 7, 8])
b = np.array([1, 2, 3, 4, 5, 6, 7, 8])
"""

stmt_mod = "a = (a + b) % 2"

# Time the bitwise AND operation
time_and = timeit.timeit(stmt=stmt_and, setup=setup_and, number=1000000)
print(f"Bitwise XOR operation time: {time_and:.6f} seconds")

# Time the addition followed by modulo operation
time_mod = timeit.timeit(stmt=stmt_mod, setup=setup_mod, number=1000000)
print(f"Addition followed by modulo operation time: {time_mod:.6f} seconds")

Bitwise XOR operation time: 0.357372 seconds
Addition followed by modulo operation time: 1.515845 seconds


In [3]:
import galois
import numpy as np
import timeit

# Define the finite field GF(2)
GF2 = galois.GF(2)

# Create matrices in GF(2)
A_gf = GF2([[1, 0, 1], [0, 1, 1], [1, 1, 0]])
B_gf = GF2([[0, 1, 0], [1, 0, 1], [0, 1, 1]])

# Matrix multiplication in GF(2)
def gf2_matrix_multiplication():
    return A_gf @ B_gf

# Benchmark the GF(2) matrix multiplication
time_gf2 = timeit.timeit(gf2_matrix_multiplication, number=100000)
print(f"GF(2) matrix multiplication time: {time_gf2:.6f} seconds")

# Create matrices as integers
A_int = np.array([[1, 0, 1], [0, 1, 1], [1, 1, 0]], dtype=int)
B_int = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 1]], dtype=int)

# Matrix multiplication followed by modulo 2
def int_matrix_multiplication_mod2():
    return (A_int @ B_int) % 2

# Benchmark the integer matrix multiplication followed by modulo 2
time_int_mod2 = timeit.timeit(int_matrix_multiplication_mod2, number=100000)
print(f"Integer matrix multiplication followed by modulo 2 time: {time_int_mod2:.6f} seconds")

GF(2) matrix multiplication time: 2.049155 seconds
Integer matrix multiplication followed by modulo 2 time: 0.204563 seconds


In [4]:
import numpy as np
import timeit

# Define the symplectic inner product function
def symplectic_inner_product(u: np.ndarray, v: np.ndarray) -> int:
    n = len(u) // 2
    return (((u[0:n] @ v[n:]) - (u[n:] @ v[0:n])) % 2)

# Define the beta function with the check
def beta_with_check(u: np.ndarray, v: np.ndarray) -> int:
    n = int(len(u) / 2)
    # check if paulis commute:
    if symplectic_inner_product(u, v) != 0:
        raise ValueError("Pauli elements must commute.")
    else:
        return (int(((u[n:] @ v[0:n]) - (u[0:n] @ v[n:])) / 2) % 2)

# Define the beta function without the check
def beta_without_check(u: np.ndarray, v: np.ndarray) -> int:
    n = int(len(u) / 2)
    return (int(((u[n:] @ v[0:n]) - (u[0:n] @ v[n:])) / 2) % 2)

# Create sample input data
u = np.random.randint(0, 2, size=10)
v = u


# Measure the execution time of beta_with_check
time_with_check = timeit.timeit(lambda: beta_with_check(u, v), number=100000)
print(f"Execution time with check: {time_with_check:.6f} seconds")

# Measure the execution time of beta_without_check
time_without_check = timeit.timeit(lambda: beta_without_check(u, v), number=100000)
print(f"Execution time without check: {time_without_check:.6f} seconds")

# Calculate the overhead
overhead = time_with_check - time_without_check
print(f"Overhead introduced by the check: {overhead:.6f} seconds")

Execution time with check: 0.545182 seconds
Execution time without check: 0.257606 seconds
Overhead introduced by the check: 0.287576 seconds


In [22]:
def symplectic_inner_product(u: np.ndarray, v: np.ndarray) -> int:
    n = len(u) // 2
    return (((u[0:n] @ v[n:]) - (u[n:] @ v[0:n])) % 2)

def symplectic_inner_product_v2(u: np.ndarray, v: np.ndarray) -> int:
    n = len(u) // 2
    return np.bitwise_xor.reduce(u[0:n] & v[n:]) ^ np.bitwise_xor.reduce(u[n:] & v[0:n])

def symplectic_inner_product_v3(u: np.ndarray, v: np.ndarray) -> int:
    n = len(u) // 2
    return (((u[0:n].dot(v[n:])) - (u[n:].dot(v[0:n]))) % 2)

def symplectic_inner_product_v4(u: np.ndarray, v: np.ndarray) -> int:
    """
    Computes the symplectic inner product of two vectors u and v in :math:`\mathbb{Z}_2`.

    Args:
        u (np.ndarray): The first vector.
        v (np.ndarray): The second vector.

    Returns:
        int: The symplectic inner product of u and v in :math:`\mathbb{Z}_2`.
    """
    assert u.shape == v.shape
    assert u.shape[0] % 2 == 0
    n = len(u) // 2
    ux, uz = u[:n], u[n:]
    vx, vz = v[:n], v[n:]
    return (uz.dot(vx) - ux.dot(vz)) % 2

# Timing comparison
def time_function(func, u, v, number=100000):
    timer = timeit.Timer(lambda: func(u, v))
    return timer.timeit(number=number)

# Test Case
u = np.random.randint(0, 2, size=30)
v = np.random.randint(0, 2, size=30)

# Time the original function
time_original = time_function(symplectic_inner_product, u, v)
print(f"Original function time: {time_original:.6f} seconds")

# Time the first optimized function
time_v2 = time_function(symplectic_inner_product_v2, u, v)
print(f"First optimized function time: {time_v2:.6f} seconds")

# Time the second optimized function
time_v3 = time_function(symplectic_inner_product_v3, u, v)
print(f"Second optimized function time: {time_v3:.6f} seconds")

# Time the third optimized function
time_v4 = time_function(symplectic_inner_product_v4, u, v)
print(f"Third optimized function time: {time_v4:.6f} seconds")


<>:14: SyntaxWarning: invalid escape sequence '\m'
<>:14: SyntaxWarning: invalid escape sequence '\m'
/var/folders/vb/d0tsm20528v3w63nm9bgvc9h0000gn/T/ipykernel_73884/1172123708.py:14: SyntaxWarning: invalid escape sequence '\m'
  """


Original function time: 0.245075 seconds
First optimized function time: 0.384503 seconds
Second optimized function time: 0.190406 seconds
Third optimized function time: 0.211302 seconds


In [23]:
import timeit

def create_matrix_with_loops():
    A = np.zeros([50, 50], np.int64)
    for i in range(50):
        for j in range(50):
            if i >= j:
                A[i, j] = 1
            else:
                A[i, j] = 0
    return A

def create_matrix_with_optimized_loops():
    A = np.zeros((50, 50), np.int64)
    for i in range(50):
        A[i, :i+1] = 1

def create_matrix_tril():
    return np.tril(np.ones((50, 50), dtype=int))

# Measure the execution time of the original nested loops implementation
time_loops = timeit.timeit(create_matrix_with_loops, number=100000)
print(f"Original nested loops implementation time: {time_loops:.6f} seconds")

# Measure the execution time of the optimized nested loops implementation
time_optimized_loops = timeit.timeit(create_matrix_with_optimized_loops, number=100000)
print(f"Optimized nested loops implementation time: {time_optimized_loops:.6f} seconds")

# Measure the execution time of the optimized numpy implementation
time_tril = timeit.timeit(create_matrix_tril, number=100000)
print(f"Optimized numpy implementation time: {time_tril:.6f} seconds")

Original nested loops implementation time: 29.455326 seconds
Optimized nested loops implementation time: 2.193832 seconds
Optimized numpy implementation time: 0.965987 seconds


In [10]:
create_matrix_with_loops()

array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]])

In [11]:
create_matrix_with_optimized_loops()

array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]])

In [31]:
a = np.array([1, 1, 1, 0], dtype=np.bool_)
print(a)

[ True  True  True False]


In [32]:
def symplectic_inner_product(u: np.ndarray, v: np.ndarray) -> int:
    """Computes the symplectic inner product of vectors u and v over GF(2)."""
    n = len(u) // 2
    ux, uz = u[:n], u[n:]
    vx, vz = v[:n], v[n:]
    return (uz.dot(vx) - ux.dot(vz)) % 2

def beta(u: np.ndarray, v: np.ndarray, skip_commutation_check=False) -> int:
    n = len(u) // 2

    if not skip_commutation_check:
        assert symplectic_inner_product(u, v) == 0

    ux, uz = u[:n], u[n:]
    vx, vz = v[:n], v[n:]

    x_terms = (ux + vx) % 2
    z_terms = (uz + vz) % 2

    u_phase = ux.dot(uz) % 4
    v_phase = vx.dot(vz) % 4
    combined_phase = x_terms.dot(z_terms) % 4

    gamma = (u_phase + v_phase + 2 * uz.dot(vx) - combined_phase) % 4
    beta = gamma // 2

    return beta

In [33]:
a.dot(a)

True

In [48]:
# create random 4x4 matrix
np.random.seed(0)
A = np.random.randint(0, 2, size=(4, 4))
B = np.random.randint(0, 2, size=(4, 4))

b = np.array([1, 1, 1, 0], dtype=np.int8)

c = A @ B @ b
print(c)

[4 6 4 2]


In [50]:
print(c[:2])

[4 6]
